## GEOSeMOSYS - exploring linear programming geospatial energy modelling 

Nandi Moksnes, Mark Howells, ...

This Jupyter Notebook is dectription of how to reproduce the results in the paper:
For any questions please contact Nandi Moksnes: nandi@kth.se

## 1. The files in "GIS_data" are downloaded and placed in a "temp" folder.

In [ ]:
from Download_files import *
import os
current = os.getcwd()
os.chdir(current)
print(os.getcwd())

download_url_data('input_data/GIS_URL.txt', 'temp')
unzip_all('input_data/GIS_URL.txt')

## 2. The files are then projected and clipped to the administrative boundaries.

In [ ]:
from Project_GIS import *
os.chdir(current)

main('../GIS_data')

## 3. Through QGIS make raster to point layer and save

1) masked_UMT37S_ken_ppp_2018_1km_Aggregated: SAGA raster to point 

This step is downloaded from zenodo so it is not needed for the user to execute.


In [ ]:
from Download_files import *
import os
current = os.getcwd()
os.chdir(current)
print(os.getcwd())
download_url_data('input_data/zenodo.txt', '/Projected_files')

## 4. The GIS layers are prepared to for a heuristic approximation for electrified settlements

In [ ]:
import matplotlib.pyplot as plt
from settlement_build import *
current = os.getcwd()
os.chdir(current)

pop_shp = '../Projected_files/raster_to_point_UMT37S_Kenya.shp'
Projected_files_path = '../Projected_files'

rasterize = raster_proximity(Projected_files_path)
points = raster_to_point(rasterize, pop_shp, Projected_files_path)

points.sample

## 5. Approximate location of urban settlements and the electrified settlements 1kmx1km resolution

In [ ]:
from elec_start import *
import os
current = os.getcwd()
os.chdir(current)

elec_actual = 0.75  # percent
pop_cutoff = 4000  # people
dist_minigrid = 5000  # meters
dist_to_trans = 5000  # meters
dist_to_sub = 5000 #meters
dist_minig = 5000 #meters
min_night_lights = 20
max_grid_dist = 20000  # meters
max_road_dist = 300  # meters
pop_cutoff2 = 36000 # people
urban_elec_ratio = 83.5  # percent
rural_elec_ratio = 71.5  # percent
pop_actual = 52570000  # peolpe
urban = 0.275  # percent
urban_cutoff = 20000
start_year = 2018
settlement = gpd.read_file("../Projected_files/settlements.shp")

settlements = pd.DataFrame(settlement, copy=True)
urbansettlements = calibrate_pop_and_urban(settlements, pop_actual, urban, urban_cutoff)
elec_current_and_future(urbansettlements, elec_actual, pop_cutoff, dist_to_trans, dist_to_sub, dist_minig, min_night_lights,
                            max_grid_dist, urban_elec_ratio, rural_elec_ratio, max_road_dist, pop_actual, pop_cutoff2, start_year)

%matplotlib inline
settlements.plot(column='elec', cmap=None, legend=True, figsize=(20, 20))

## 6. Download Renewable.ninja data for all 378 locations

In [ ]:
from renewable_ninja_download import *

shapefile = '../Projected_files/new_40x40points_WGSUMT37S.shp'
path = "temp"
coordinates = project_vector(shapefile)
wind, solar = csv_make(coordinates)
down = download(path, wind, solar)



## 7. Scale down from1x1 km to 40 x 40 km demand cells